In [1]:
def rcMax(m1, row, col):
    vertical = 0
    horizontal = 0
    
    n = len(m1)
    
    for i in range(n):
        if (m1[row][i] == 0):
            horizontal += 1
            
    for i in range(n):
        if (m1[i][col] == 0):
            vertical += 1
            
    if vertical > horizontal:
        return vertical
    else:
        return horizontal * -1
    
def clearNeighbours(m2, m3, row, col):
    
    n = len(m2)
    
    if m2[row][col] > 0:
        for i in range(n):
            if m2[i][col] > 0:
                m2[i][col] = 0
            m3[i][col] += 1
    else:
        for i in range(n):
            if (m2[row][i] < 0):
                m2[row][i] = 0
            m3[row][i] += 1
            
    #m2[row][col] = 0
    #m3[row][col] = 1
    
def printMat(m):
    n = len(m)
    
    for row in range(n):
        string = ''
        for col in range(n):
            string += (str(m[row][col]) + '\t')
        print(string)
    print('\n')
    
def coverZeros(m1):
    
    n = len(m1)
    
    m2 = [[0 for j in range(n)] for i in range(n)]
    m3 = [[0 for j in range(n)] for i in range(n)]
    
    for row in range(n):
        for col in range(n):
            if m1[row][col] == 0:
                m2[row][col] = rcMax(m1, row, col)
    
    #printMat(m2)
    
    for row in range(n):
        for col in range(n):
            if abs(m2[row][col]) > 0:
                clearNeighbours(m2, m3, row, col)
             
    #printMat(m3)        
    return m3

def getLineCount(m):
    n = len(m)
    
    line_cnt = 0

    for col in range(n):
        is_column = True
        for row in range(n):
            if m[row][col] == 0:
                is_column = False
                break
        if not is_column:
            continue
        line_cnt += 1

    for row in range(n):
        is_row = True
        for col in range(n):
            if m[row][col] == 0:
                is_row = False
                break
        if not is_row:
            continue
        line_cnt += 1
        
    return line_cnt


def findMin(c_mat, mask=None):
    row_count = len(c_mat)
    col_count = len(c_mat[0])
    
    min_elem = float('inf')
    
    if mask is not None:
        for i in range(row_count):
            for j in range(col_count):
                if mask[i][j]:
                    continue
                if c_mat[i][j] < min_elem:
                    min_elem = c_mat[i][j]
                    min_pos = (i, j)
    else:
        for i in range(row_count):
            for j in range(col_count):
                if c_mat[i][j] < min_elem:
                    min_elem = c_mat[i][j]
                    min_pos = (i, j)
                    
    return min_pos

def hungarian(c_mat):
    
    row_count = len(c_mat)
    col_count = len(c_mat[0])
    
    for i in range(row_count):
        row_min = float('inf')
        for j in range(col_count):
            if c_mat[i][j] < row_min:
                row_min = c_mat[i][j]
        for j in range(col_count):
            c_mat[i][j] -= row_min
    
    for i in range(col_count):
        col_min = float('inf')
        for j in range(row_count):
            if c_mat[j][i] < col_min:
                col_min = c_mat[j][i]
        for j in range(row_count):
            c_mat[j][i] -= col_min
    
    lines = coverZeros(c_mat)
    lineCount = getLineCount(lines)
    
    if lineCount == row_count:
        return c_mat
    
    while (lineCount != row_count):
        min_idx = findMin(c_mat, mask=lines)
        min_elem = c_mat[min_idx[0]][min_idx[1]]
        
        print('OK')
        
        print('OK')
        print(c_mat)
        print(lines)
        print('\n')
        for i in range(row_count):
            for j in range(col_count):
                c_mat[i][j] += min_elem * (lines[i][j] - 1)

        lines = coverZeros(c_mat)
        lineCount = getLineCount(lines)
  
    return c_mat

In [2]:

c_mat = [
    [10, 19, 8, 15, 19],
    [10, 18, 7, 17, 19],
    [13, 16, 9, 14, 19],
    [12, 19, 8, 18, 19],
    [14, 17, 10, 19, 19]
]

'''
c_mat = [
    [100, 200, 350],
    [350, 250, 400],
    [150, 300, 450]
]
'''

hungarian(c_mat)

OK
OK
[[0, 4, 0, 2, 2], [1, 4, 0, 5, 3], [2, 0, 0, 0, 1], [2, 4, 0, 5, 2], [2, 0, 0, 4, 0]]
[[1, 1, 2, 1, 1], [0, 0, 1, 0, 0], [1, 1, 2, 1, 1], [0, 0, 1, 0, 0], [1, 1, 2, 1, 1]]




[[0, 4, 1, 2, 2],
 [0, 3, 0, 4, 2],
 [2, 0, 1, 0, 1],
 [1, 3, 0, 4, 1],
 [2, 0, 1, 4, 0]]